# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import qgrid
import yaml

import ipywidgets as widgets
from pathlib import Path

from interface import DupInterface, layouts
import table

# QGrid

In [ ]:
di = DupInterface.from_yaml(r'jsl\jsl.yaml')
di.loader.children[1].value = r'./jsl/df.pkl'
display(di.widget)